In [1]:
! pip install pandas
! pip install requests
! pip install plyer 


In [14]:
import pandas as pd
import requests as rq
import plyer 
import datetime
import sqlite3

In [3]:
def error(error_message):
    current_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    message = f'{error_message} {current_date}'
    plyer.notification.notify(
        title='Error',
        message=message,
        app_name="Alerta"
    )

In [4]:
def get_response(url):
    response = rq.get(url)
    if response.status_code == 200:
        data_json = response.json()
        return data_json
    else:
        error('Tabela nao encontrada')

In [5]:
def get_tables(url):
    response = get_response(url)
    return response

In [6]:
def create_df(data):
    if data is None:
        return pd.DataFrame()
    df = pd.DataFrame(data)
    df['country'] = df['name'].apply(lambda x: x.get('common', ''))
    df = df[['country', 'capital', 'area', 'population']]
    return df

In [7]:
def filter_df(df, min_population):
    return df[df['population'] > min_population]

In [8]:
def unstack_df(df, index_col):
    return df.set_index(index_col).unstack()

In [20]:
def create_table_sqlite(data, nome_tabela):
    # Converter a série em um DataFrame
    df = pd.DataFrame(data)

    # Converter o nome das colunas para strings
    df.columns = df.columns.astype(str)

    # Conectar ao banco de dados SQLite
    conn = sqlite3.connect('dados_paises.db')
    cursor = conn.cursor()

    # Criar a tabela no banco de dados
    colunas_str = ', '.join([f'"{coluna}"' for coluna in df.columns])
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {nome_tabela} ({colunas_str})")

    # Inserir os dados do DataFrame na tabela
    for index, row in df.iterrows():
        values = ', '.join([f'"{str(row[coluna])}"' for coluna in df.columns])
        cursor.execute(f"INSERT INTO {nome_tabela} VALUES ({values})")

    # Commitar as alterações e fechar a conexão
    conn.commit()
    conn.close()
    
    # Exemplo de uso da função para criar e preencher a tabela SQLite


In [21]:
def main():
    urls = {
        'Lang': 'https://restcountries.com/v3.1/lang/portuguese?limit=14',
        'Region': 'https://restcountries.com/v3.1/region/europe',
        'Subregion': 'https://restcountries.com/v3.1/subregion/Caribbean'
    }

    for key, url in urls.items():
        data = get_response(url)
        df = create_df(data)
        print(f"{key} DataFrame:")
        print(df)
        
        df_filtered = filter_df(df, 500000)
        print(f"{key} DataFrame (filtered):")
        print(df_filtered)
        
        df_unstacked = unstack_df(df_filtered, 'country')
        print(f"{key} DataFrame (unstacked):")
        print(df_unstacked)
        print()

        # Chamar a função para criar e preencher a tabela SQLite
        create_table_sqlite(df_filtered, key)
        
main()

Lang DataFrame:
                 country     capital       area  population
0             Mozambique    [Maputo]   801590.0    31255435
1               Portugal    [Lisbon]    92090.0    10305564
2          Guinea-Bissau    [Bissau]    36125.0     1967998
3  São Tomé and Príncipe  [São Tomé]      964.0      219161
4                 Brazil  [Brasília]  8515767.0   212559409
5            Timor-Leste      [Dili]    14874.0     1318442
6                  Macau         NaN       30.0      649342
7      Equatorial Guinea    [Malabo]    28051.0     1402985
8                 Angola    [Luanda]  1246700.0    32866268
9             Cape Verde     [Praia]     4033.0      555988
Lang DataFrame (filtered):
             country     capital       area  population
0         Mozambique    [Maputo]   801590.0    31255435
1           Portugal    [Lisbon]    92090.0    10305564
2      Guinea-Bissau    [Bissau]    36125.0     1967998
4             Brazil  [Brasília]  8515767.0   212559409
5        Timor-Le